In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils import seed_random_generators
seed_random_generators()

In [3]:
from pathlib import Path
from npz_loader import load_episodes

dataset_root = Path("dataset")
holdout_game_dirs = [
    dataset_root.joinpath("BeamRiderNoFrameskip-v4", "BeamRiderNoFrameskip-v4"),
    # dataset_root.joinpath("BreakoutNoFrameskip-v4", "BreakoutNoFrameskip-v4")
]

main_game_dirs = [
    dataset_root.joinpath("EnduroNoFrameskip-v4", "EnduroNoFrameskip-v4"),
    dataset_root.joinpath("MsPacmanNoFrameskip-v4", "MsPacmanNoFrameskip-v4"),
    # dataset_root.joinpath("PongNoFrameskip-v4", "PongNoFrameskip-v4"),
    # dataset_root.joinpath("QbertNoFrameskip-v4", "QbertNoFrameskip-v4"),
    # dataset_root.joinpath("SeaquestNoFrameskip-v4", "SeaquestNoFrameskip-v4"),
    # dataset_root.joinpath("SpaceInvadersNoFrameskip-v4", "SpaceInvadersNoFrameskip-v4")
]
episodes = load_episodes(main_game_dirs, holdout_game_dirs)

Loaded 880 episodes


In [4]:
from utils import sample_list
sampled_episodes = sample_list(episodes, fraction=0.10)

Sampled 88 items (10.0% of 880 total)


In [5]:
from epsiode_dataloader import make_train_val_dataloaders

main_bundle, holdout_bundle, bins = make_train_val_dataloaders(
    episodes=sampled_episodes,
    holdout_game_dirs=holdout_game_dirs,
    train_frac=0.8,
    timestep_window_size=4,
)

In [6]:
base_dir = Path("output")

# Baseline

In [ ]:
# Kenny started this

# Freeze

In [ ]:
from experiment_freeze import experiment_freeze
from mgdt_model import Freezeable

experiment_freeze("Freeze Transformer", main_bundle, holdout_bundle, bins, [Freezeable.Transformer], base_dir.joinpath("freeze_transformer"), "Frozen Transformer")

[I 2025-12-08 19:03:54,294] A new study created in memory with name: no-name-e261611d-3af5-4272-b7ea-72f142b94eef


Trial params: {'lr': 4.2695830456224604e-05, 'emb_size': 1024, 'n_layers': 5, 'n_heads': 4, 'num_epochs': 3, 'max_timestep_window_size': 11}


Epoch 3/3:  80%|███████▉  | 1582/1984 [01:58<00:23, 17.42it/s]

In [ ]:
experiment_freeze("Freeze Obs Encoder", main_bundle, holdout_bundle, bins, [Freezeable.ObsEncoder], base_dir.joinpath("freeze_obs_encoder"), "Frozen Obs Encoder")